In [ ]:
!pip install spectral
import scipy.io as sio
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import spectral
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import numpy as np
from sklearn.decomposition import FactorAnalysis
import pandas as pd
from sklearn.decomposition import KernelPCA 
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import locally_linear_embedding
from sklearn.manifold import MDS
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from operator import truediv


In [ ]:
# Dataset
X = sio.loadmat('DATASETS/Indian_pines_corrected.mat')['indian_pines_corrected']
y = sio.loadmat('DATASETS/Indian_pines_gt.mat')['indian_pines_gt']

In [ ]:
#remove 0 label data points
X_LLE = np.reshape(X, (-1, X.shape[2]))
newy = y.ravel()

data = []
y_new = []
for i,j in zip(newy,X_LLE):
  if (i!=0):
    y_new.append(i)
    data.append(j)
data = np.array(data)
y_new = np.array(y_new)

In [ ]:
def applyKPCA(X,kernel_, numComponents=75):
    kpca = KernelPCA(kernel =kernel_,n_components = numComponents) 
    newX = kpca.fit_transform(X)
    return newX

In [ ]:
def applyPCA(X, numComponents=75):
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(X)
    return newX,pca

In [ ]:
def applyFA(X,K):
  transformer = FactorAnalysis(n_components=K)
  X_transformed = transformer.fit_transform(X)
  return X_transformed

In [ ]:
def applyLDA(X,y,K):
  clf = LinearDiscriminantAnalysis(n_components=K)
  X_transformed = clf.fit_transform(X,y)
  return X_transformed

In [ ]:
def applyLLE(X,K,v):
  embedding = LocallyLinearEmbedding(n_components=K,n_neighbors = v)
  X_transformed = embedding.fit_transform(X)
  return X_transformed

In [ ]:
def applyMDS(X,K):
  embedding = embedding = MDS(n_components=K)
  X_transformed = embedding.fit_transform(X)
  return X_transformed

In [ ]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

#Gridsearch on SVM

In [ ]:
PCA_X,pca = applyPCA(data,30)

In [ ]:
#split dataset in train-test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(PCA_X, y_new, test_size=0.3, random_state=345,stratify=y_new)

In [ ]:
#paramteres for SVM
gamma=['scale','auto',0.0001,0.001,0.01,0.1,0.25,0.5,1,2,4,8,10,16,100,128,256,300,450,500]
C=[0.01,0.1,0.25,0.5,1,2,4,8,10,16,32,64,128,256,300,450]
degree = [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
fun = ['ovo','ovr']

param = {}

kernel = 'poly'

if(kernel == 'poly'):
  for i in fun:
    for j in C:
      for k in degree:
        svm = SVC(kernel=kernel,degree=k,C=j,verbose=3,fun = i)
        svm.fit(X_train, y_train)
        param[(i,j,k)] = svm.score(X_test,y_test)

elif (kernel == 'rbf'):
  for i in fun:
    for j in C:
      for k in gamma:
        svm = SVC(kernel=kernel,gamma=k,C=j,verbose=3,fun = i)
        svm.fit(X_train, y_train)
        param[(i,j,k)] = svm.score(X_test,y_test)

else:
  for i in fun:
    for j in C:
        svm = SVC(kernel=kernel,C=j,verbose=3,fun = i)
        svm.fit(X_train, y_train)
        param[(i,j,k)] = svm.score(X_test,y_test)



In [ ]:
#save gridsearch result
df = pd.DataFrame()

if(kernel in ['poly','rbf']):

  param = 'degree' if kernel=='poly' else 'gamma'

  list1 = [i[0] for i in list(param.keys())]
  list2 = [i[1] for i in list(param.keys())]
  list3 = [i[2] for i in list(param.keys())]
  values = [i for i in list(param.values())]

  df['fun'] = list1
  df['C'] = list2
  df[param] = list3
  df['score'] = values
  df.to_csv('/content/drive/My Drive/project/indiana_pines/Grid_LLE/CSV/filename.csv', index=False)

else:

  list1 = [i[0] for i in list(param.keys())]
  list2 = [i[1] for i in list(param.keys())]
  values = [i for i in list(param.values())]

  df['fun'] = list1
  df['C'] = list2
  df['score'] = values
  df.to_csv('/content/drive/My Drive/project/indiana_pines/Grid_LLE/CSV/filename.csv', index=False)

In [ ]:
def get_key(val): 
    for key, value in my_dict.items(): 
         if val == value: 
             return key 
  
    return "key doesn't exist"

In [ ]:
#assign SVM best SVM parameter from gridsearch result

max_accuracy = max(list(param.values()))
best_param = get_key(max_accuracy)

if(kernel == 'poly'):
  clf = SVC(kernel='poly', fun = best_param[0],C = best_param[1], degree = best_param[2]).fit(X_train,y_train)

elif(kernel == 'rbf'):
  clf = SVC(kernel='rbf', fun = best_param[0],C = best_param[1], gamma = best_param[2]).fit(X_train,y_train)

else:
  clf = SVC(kernel='linear', fun = best_param[0],C = best_param[1]).fit(X_train,y_train)

y_pred = svm.predict(X_test)

In [ ]:
confusion = confusion_matrix(y_test, y_pred)
each_acc, aa = AA_andEachClassAccuracy(confusion)
kappa = cohen_kappa_score(y_test, y_pred)

print("classwise accuracy")
print(each_acc)

print("kappa :",kappa)
print("aa :",aa)
print("accuracy score:",accuracy_score(y_test,y_pred))

In [ ]:
#visualizing output
y_predict = SVM.predict(PCA_X)
outputs = np.zeros((145,145))

count = 0
for i in range(145):
  for j in range(145):
    if(int(y[i,j])==0):
      continue
    else:
      outputs[i][j] = int(y_predict[count])
      count = count + 1

predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))